In [1]:
from pytest import approx, fixture
import larix as lx
import jax.numpy as jnp
import numpy as np
import xarray as xr
import jax

def mtc7():
    d = lx.examples.MTC()
    y = d.icase[:7]
    y['groupid'] = xr.DataArray([1,1,1,2,2,2,2], dims=d.dc.CASEID)
    return y


def model7():
    d = mtc7()
    m = lx.Model(d)
    P, X, PX = lx.P, lx.X, lx.PX
    m.utility_co[2] = P("ASC_SR2") + P("hhinc#2") * X("hhinc")
    m.utility_co[3] = P("ASC_SR3P") + P("hhinc#3") * X("hhinc")
    m.utility_co[4] = P("ASC_TRAN") + P("hhinc#4") * X("hhinc")
    m.utility_co[5] = P("ASC_BIKE") + P("hhinc#5") * X("hhinc")
    m.utility_co[6] = P("ASC_WALK") + P("hhinc#6") * X("hhinc")
    m.utility_ca = PX("tottime") + PX("totcost")
    m.availability_var = 'avail'
    m.choice_ca_var = 'chose'
    m.title = "MTC Example 1 (Simple MNL)"
    m.mix_parameter("tottime", "tottime_s")
    m.groupid = 'groupid'
    m.n_draws = 5000
    return m

m = model7()

/Users/jeffnewman/LocalGit/larix/larix/model/numbamodel.py:23: UserWarning: 

##### larix is experimental, and not feature-complete  #####
the first time you import on a new system, this package will
compile optimized binaries for your machine, which may take 
a little while, please be patient 

  warnings.warn( ### EXPERIMENTAL ### )
OMP: Info #273: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [2]:
m._make_random_draws(n_draws=m.n_draws)

In [3]:
def _get_jnp_array(dataset, name):
    if name not in dataset:
        return None
    return jnp.asarray(dataset[name])


co = _get_jnp_array(m.dataset, 'co')
ca = _get_jnp_array(m.dataset, 'ca')
av = _get_jnp_array(m.dataset, 'av')
ch = _get_jnp_array(m.dataset, 'ch')


bund = dict(ca=ca, co=co, ch=ch, av=av)
bund0 = dict(ca=ca[0], co=co[0], ch=ch[0], av=av[0])

gbund = dict(rk=jax.random.PRNGKey(123))

In [4]:
m._jax_loglike(m.pvals, bund0, dict(rk=jax.random.PRNGKey(122)), 1059)

DeviceArray(-4.604921, dtype=float32)

In [5]:
m.jax_loglike(m.pvals)

DeviceArray([-4.60493, -6.17343], dtype=float32)

In [6]:
m.jax_d_loglike_total(m.pvals)

DeviceArray([ -1.016548,  -0.516912,  -1.517042,   1.483725,  -0.36627 ,  17.152428, -70.35146 ,  89.67974 ,
             -53.453907, -19.563515,  64.6979  , -71.15323 ,   1.194303], dtype=float32)

In [10]:
%timeit m.jax_d_loglike_total(m.pvals)

21.8 ms ± 520 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
%timeit m.jax_loglike(m.pvals)

9.86 ms ± 69.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [8]:
m.n_draws = 5000

In [9]:
stop

NameError: name 'stop' is not defined

In [ ]:
jax.random.split(jax.random.PRNGKey(123), (5,5))

In [ ]:
xr.DataArray(jax.random.split(jax.random.PRNGKey(123), 5), dims=('c5','two'))

In [ ]:
jax.vmap(jax.random.split, in_axes=(0, None))(jax.random.PRNGKey(123), 5)

In [ ]:
jax.vmap(jax.random.split, in_axes=(0, None))(
    jax.random.split(jax.random.PRNGKey(123), 3),
    5
).shape

In [ ]:
def onekey(key, n):
    return key

jax.random.PRNGKey(123).ndim

In [ ]:
rk = jax.random.split(jax.random.PRNGKey(123), 1)


In [ ]:
vsplit = jax.vmap(jax.random.split, in_axes=(0, None))

In [ ]:
def ssplit(key, shapes):
    if key.ndim == 1:
        new_keys = jax.random.split(key, shapes[0])
    else:
        new_keys = jax.vmap(jax.random.split, in_axes=(0, None))(key, shapes[0])
    return new_keys, shapes[1:]

k, s = jax.random.PRNGKey(123), [5,3]

k, s = ssplit(k, s)
k, s = ssplit(k, s)
k, s

In [ ]:
def vsplit(key_array, shapes):
    new_keys = jax.vmap(
        jax.random.split, in_axes=(0, None)
    )(key_array, shapes[0])
    return new_keys, shapes[1:]

In [ ]:
vsplit(jax.random.PRNGKey(123), [5,3])

In [ ]:
vsplit = jax.vmap(vsplit, in_axes=(0, None))

In [ ]:
rk1 = vsplit(rk, 5)


In [ ]:
jax.vmap(jax.random.split, in_axes=(0, None))(
    jax.random.split(jax.random.PRNGKey(123), 1),
    5
).shape

In [ ]:
jax.vmap(m._jax_loglike, in_axes=(None, 0))(m.pvals, bund)

In [ ]:
m.pvals = 'null'
m.pvals = {'tottime': -1}

m.jax_loglike(m.pvals)

In [ ]:
m.jax_loglike_casewise(m.pvals)

In [ ]:
lx.examples.MTC().icase[:7]['chose']

In [ ]:
m.pvals = 'null'
m.pvals = {"hhinc#5": -1}

In [ ]:
np.asarray(m.jax_utility(m.pvals))

In [ ]:
m.datatree.root_dataset['tottime'].to_numpy()

In [ ]:
inf = np.inf
np.array([
    [  0.,   0.,   0.,   0.,   0., -inf,   0.],
    [  0.,   0.,   0.,   0.,   0., -inf,   0.],
    [  0.,   0.,   0.,   0., -inf, -inf,   0.],
    [  0.,   0.,   0.,   0., -inf, -inf,   0.],
    [-inf,   0.,   0.,   0.,   0., -inf,   0.],
], dtype=np.float32)